In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

tf.__version__

'2.15.0'

# Softmax classification

* 임의의 Dataset 준비
* 3개의 클래스로 분류할 데이터 준비


In [ ]:
x_data = [[1., 2., 1., 1.],
          [2., 1., 3., 2.],
          [3., 1., 3., 4.],
          [4., 1., 5., 5.],
          [1., 7., 5., 5.],
          [1., 2., 5., 6.],
          [1., 6., 6., 6.],
          [1., 7., 7., 7.]] # 8x4
y_data = [[0., 0., 1.],
          [0., 0., 1.],
          [0., 0., 1.],
          [0., 1., 0.],
          [0., 1., 0.],
          [0., 1., 0.],
          [1., 0., 0.],
          [1., 0., 0.]] # 8x3

x_test = [[1., 1., 1., 1.]]
y_test = [[0., 0., 1.]]


## 임의의 Data를 이용해서 3개의 클래스를 가지는 데이터셋 생성

In [ ]:
#dataset을 선언합니다.
dataset = tf.data.Dataset.from_tensor_slices((x_data, y_data))
dataset = dataset.repeat(1).batch(8)

nb_classes = 3 # class의 개수입니다.

print(tf.Variable(x_data))
print(tf.Variable(y_data))

<tf.Variable 'Variable:0' shape=(8, 4) dtype=float32, numpy=
array([[1., 2., 1., 1.],
       [2., 1., 3., 2.],
       [3., 1., 3., 4.],
       [4., 1., 5., 5.],
       [1., 7., 5., 5.],
       [1., 2., 5., 6.],
       [1., 6., 6., 6.],
       [1., 7., 7., 7.]], dtype=float32)>
<tf.Variable 'Variable:0' shape=(8, 3) dtype=float32, numpy=
array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.]], dtype=float32)>


In [ ]:
#Weight and bias setting
W = tf.Variable(tf.random.normal([4, nb_classes]), name='weight')
b = tf.Variable(tf.random.normal([nb_classes]), name='bias')
variables = [W, b]

tf.print(W,b)

[[-0.781559408 0.213658348 0.562138617]
 [-1.23078358 -0.256780028 0.584250093]
 [-2.12305093 0.0466100127 -0.740654945]
 [1.42141116 1.5697372 -0.903915167]] [1.14577115 0.240724072 -0.71792388]


# 가설 설정

* 가설에서 예측한 값들을 이용해 예측값들을 확률로 표현한다.

## $$ y_k = \frac{exp(x_k)}{\sum_{i=1}^{n}exp(x_i)}  $$

In [ ]:
# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
def hypothesis_softmax(X):
    return tf.nn.softmax(tf.matmul(X, W) + b)

tf.print(hypothesis_softmax(tf.Variable(x_data)))

[[0.0114036966 0.889004707 0.0995916]
 [0.000141054858 0.99866569 0.00119322923]
 [3.88055378e-05 0.999949038 1.20236891e-05]
 ...
 [1.03984314e-06 0.999998868 2.04282387e-08]
 [2.41367903e-09 0.999999702 2.68748153e-07]
 [8.97400626e-11 0.99999994 2.39002347e-08]]


## 가설을 검증할 Cross Entropy 함수를 정의합니다

$$
\begin{align}
cost(h(x),y) & = −\sum_{n=1}^{n} Y log(h(x))
\end{align}
$$

In [ ]:
def loss_fn(hypothesis, labels):
    loss = tf.keras.losses.categorical_crossentropy(labels, hypothesis)
    # tf.keras.losses.binary_crossentropy(labels, hypothesis) # 이진분류용 CE Loss
    return loss

optimizer = tf.compat.v1.train.GradientDescentOptimizer(learning_rate=0.01)

### 학습 진행

In [ ]:
epochs = 5000

for step in range(epochs):
  for features, labels in dataset:
    with tf.GradientTape() as tape:
      loss_value = loss_fn(hypothesis_softmax(features),labels)
      grads = tape.gradient(loss_value, [W,b])
      optimizer.apply_gradients(grads_and_vars=zip(grads,[W,b]))
      if step % 100 == 0:
            print("Iter: {}, Loss: {:.4f}".format(step, tf.reduce_mean(loss_value.numpy()).numpy()))

Iter: 0, Loss: 6.5753
Iter: 100, Loss: 0.6299
Iter: 200, Loss: 0.5568
Iter: 300, Loss: 0.5079
Iter: 400, Loss: 0.4675
Iter: 500, Loss: 0.4322
Iter: 600, Loss: 0.4029
Iter: 700, Loss: 0.3790
Iter: 800, Loss: 0.3577
Iter: 900, Loss: 0.3384
Iter: 1000, Loss: 0.3210
Iter: 1100, Loss: 0.3052
Iter: 1200, Loss: 0.2907
Iter: 1300, Loss: 0.2774
Iter: 1400, Loss: 0.2652
Iter: 1500, Loss: 0.2539
Iter: 1600, Loss: 0.2435
Iter: 1700, Loss: 0.2339
Iter: 1800, Loss: 0.2249
Iter: 1900, Loss: 0.2165
Iter: 2000, Loss: 0.2087
Iter: 2100, Loss: 0.2015
Iter: 2200, Loss: 0.1946
Iter: 2300, Loss: 0.1882
Iter: 2400, Loss: 0.1822
Iter: 2500, Loss: 0.1766
Iter: 2600, Loss: 0.1712
Iter: 2700, Loss: 0.1662
Iter: 2800, Loss: 0.1614
Iter: 2900, Loss: 0.1569
Iter: 3000, Loss: 0.1526
Iter: 3100, Loss: 0.1486
Iter: 3200, Loss: 0.1447
Iter: 3300, Loss: 0.1410
Iter: 3400, Loss: 0.1375
Iter: 3500, Loss: 0.1342
Iter: 3600, Loss: 0.1310
Iter: 3700, Loss: 0.1280
Iter: 3800, Loss: 0.1251
Iter: 3900, Loss: 0.1223
Iter: 4000, 

## Sample 데이터를 넣고 테스트해봅시다.

In [ ]:
sample_data = [[2,1,3,2]] # answer_label [[0,0,1]]
sample_data = np.asarray(sample_data, dtype=np.float32)

a = hypothesis_softmax(sample_data)

print(a)
print(tf.argmax(a, 1)) #index: 2

tf.Tensor([[1.8219676e-04 4.9082153e-02 9.5073569e-01]], shape=(1, 3), dtype=float32)
tf.Tensor([2], shape=(1,), dtype=int64)


## 데이터를 이용해서 예측

In [ ]:
b = hypothesis_softmax(x_test)
print(b)
print(tf.argmax(b, 1))
print(tf.argmax(y_test, 1)) # matches with y_data


tf.Tensor([[3.459829e-07 5.923435e-04 9.994073e-01]], shape=(1, 3), dtype=float32)
tf.Tensor([2], shape=(1,), dtype=int64)
tf.Tensor([2], shape=(1,), dtype=int64)


In [ ]:
def accuracy_fn(hypothesis, labels):
    hypothesis = tf.argmax(hypothesis)
    hypothesis = tf.cast(hypothesis, dtype=tf.float32)
    predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
    labels = tf.argmax(labels)
    labels = tf.cast(labels, dtype=tf.float32)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, labels), dtype=tf.float32))
    return accuracy

test_acc = accuracy_fn(hypothesis_softmax(x_test),y_test)
print("Testset Accuracy: {:.4f}".format(test_acc))

Testset Accuracy: 1.0000
